In [9]:
#MATH161: CIFAR-10 MixUp vs CutMix with AdamW

# Setup — configs, utils, model, MixUp/CutMix
import os, csv, random, numpy as np, torch
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR
import torchvision
import torchvision.transforms as T
from torchvision.models import resnet18

# configs
def _merge(a, b):
    out = dict(a)
    for k, v in b.items():
        if isinstance(v, dict) and isinstance(out.get(k), dict):
            out[k] = _merge(out[k], v)
        else:
            out[k] = v
    return out

BASE = {
    "seed": 42,
    "epochs": 100,
    "batch_size": 128,
    "num_workers": 2,  # set 2 for Colab/low-core systems to avoid worker warning
    "amp": True,
    "label_smoothing": 0.0,
    "optimizer": {"name": "adamw", "lr": 5e-4, "weight_decay": 0.02, "betas": (0.9, 0.999), "eps": 1e-8},
    "scheduler": {"warmup_epochs": 5},
    "model": {"name": "resnet18_cifar"},
    "data": {
        "root": "./data",
        "mean": (0.4914, 0.4822, 0.4465),
        "std":  (0.2470, 0.2435, 0.2616),
    },
    "augment_mode": "off",   # off | mixup | cutmix
    "mixup_alpha": 0.0,
    "cutmix_alpha": 0.0,
    "save_dir": "results/run",
}
MIXUP  = {"augment_mode": "mixup",  "mixup_alpha": 0.4, "save_dir": "results/mixup"}
CUTMIX = {"augment_mode": "cutmix", "cutmix_alpha": 1.0, "save_dir": "results/cutmix"}

PROFILES = {"base": BASE, "mixup": _merge(BASE, MIXUP), "cutmix": _merge(BASE, CUTMIX)}

def get_config(profile: str, overrides: dict | None = None):
    if profile not in PROFILES:
        raise ValueError(f"unknown profile: {profile}")
    cfg = dict(PROFILES[profile])
    if overrides:
        cfg = _merge(cfg, overrides)
    return cfg

#  io + reproducibility
def ensure_dir(d): os.makedirs(d, exist_ok=True)
def save_ckpt(state, path): torch.save(state, path)
def csv_logger(path):
    f = open(path, "w", newline="")
    w = csv.DictWriter(f, fieldnames=["epoch","train_loss","val_loss","val_acc"])
    w.writeheader()
    return f, w

def set_seed(seed: int):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#  optimizer and scheduler
def build_optimizer(model, cfg):
    opt = cfg["optimizer"]
    if opt["name"].lower() != "adamw":
        raise ValueError("This setup uses AdamW for both MixUp and CutMix.")
    return AdamW(model.parameters(), lr=opt["lr"], weight_decay=opt["weight_decay"],
                 betas=opt.get("betas", (0.9,0.999)), eps=opt.get("eps", 1e-8))

def build_warmup_cosine(optimizer, steps_per_epoch: int, epochs: int, warmup_epochs: int):
    warmup_iters = max(1, warmup_epochs * steps_per_epoch)
    total_iters  = max(1, epochs * steps_per_epoch)
    cosine_iters = max(1, total_iters - warmup_iters)
    warmup = LinearLR(optimizer, start_factor=1e-3, end_factor=1.0, total_iters=warmup_iters)
    cosine = CosineAnnealingLR(optimizer, T_max=cosine_iters)
    return SequentialLR(optimizer, [warmup, cosine], milestones=[warmup_iters])

#  model
def build_model(name: str, num_classes=10):
    if name != "resnet18_cifar":
        raise ValueError(f"unsupported model: {name}")
    m = resnet18(weights=None, num_classes=num_classes)
    m.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    m.maxpool = nn.Identity()
    return m

#  MixUp / CutMix
def mixup_data(x, y, alpha: float):
    lam = torch.distributions.Beta(alpha, alpha).sample().item() if alpha and alpha > 0 else 1.0
    idx = torch.randperm(x.size(0), device=x.device)
    return lam * x + (1 - lam) * x[idx], y, y[idx], lam

def mixup_criterion(crit, pred, y_a, y_b, lam):
    return lam * crit(pred, y_a) + (1 - lam) * crit(pred, y_b)

def _rand_bbox(W, H, lam):
    r = (1 - lam) ** 0.5
    cw, ch = int(W*r), int(H*r)
    cx, cy = np.random.randint(W), np.random.randint(H)
    x1, y1 = np.clip(cx - cw//2, 0, W), np.clip(cy - ch//2, 0, H)
    x2, y2 = np.clip(cx + cw//2, 0, W), np.clip(cy + ch//2, 0, H)
    return x1, y1, x2, y2

def cutmix_data(x, y, alpha: float):
    lam = torch.distributions.Beta(alpha, alpha).sample().item() if alpha and alpha > 0 else 1.0
    bs, _, H, W = x.size()
    idx = torch.randperm(bs, device=x.device)
    x1, y1, x2, y2 = _rand_bbox(W, H, lam)
    x[:, :, y1:y2, x1:x2] = x[idx, :, y1:y2, x1:x2]
    lam = 1 - ((x2 - x1) * (y2 - y1)) / (W * H)  # exact area ratio
    return x, y, y[idx], lam



In [10]:
# CIFAR10 MixUp vs CutMix + AdamW
# Wire everything together: data, model, optimizer/scheduler, augmentation, AMP, training loop, validation, and logging.
# Per-iteration stepping keeps LR in sync with updates; AMP speeds up on modern GPUs with low complexity.

import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T
from tqdm import tqdm
from contextlib import nullcontext  # for AMP shim

# ---------------- AMP shim (portable across torch versions / CPU-GPU) ----------------
def make_scaler(enabled: bool):
    if torch.cuda.is_available():
        return torch.cuda.amp.GradScaler(enabled=enabled)
    # CPU: no-op scaler with same API
    class _Dummy:
        def scale(self, x): return x
        def step(self, opt): opt.step()
        def update(self): pass
    return _Dummy()

def amp_autocast(enabled: bool):
    if torch.cuda.is_available():
        return torch.cuda.amp.autocast(enabled=enabled)
    return nullcontext()

#  args
def parse_args(argv=None):
    ap = argparse.ArgumentParser()
    ap.add_argument("--profile", choices=["base","mixup","cutmix"],
                    default=os.environ.get("PROFILE","mixup"),
                    help="experiment profile; default='mixup' (override with PROFILE env var)")
    ap.add_argument("--save_dir", type=str, default=None)
    ap.add_argument("--epochs", type=int, default=None)
    ap.add_argument("--batch_size", type=int, default=None)
    ap.add_argument("--lr", type=float, default=None)
    ap.add_argument("--weight_decay", type=float, default=None)
    ap.add_argument("--mixup_alpha", type=float, default=None)
    ap.add_argument("--cutmix_alpha", type=float, default=None)
    if argv is None:  # ignore IPython/Colab argv in notebooks
        argv = []
    return ap.parse_args(argv)

#  transforms
def build_transforms(mean, std):
    train_tfms = T.Compose([
        T.RandomCrop(32, padding=4),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize(mean, std)
    ])
    test_tfms = T.Compose([
        T.ToTensor(),
        T.Normalize(mean, std)
    ])
    return train_tfms, test_tfms

#  eval
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    total = correct = 0
    loss_sum = 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss_sum += F.cross_entropy(logits, y).item() * y.size(0)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
    return loss_sum / total, correct / total

#  train one epoch
def train_one_epoch(model, loader, optimizer, scaler, device, criterion,
                    scheduler, augment_mode, mixup_alpha, cutmix_alpha, use_amp):
    model.train()
    total = 0
    loss_sum = 0.0
    for x, y in tqdm(loader, leave=False, desc="train"):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)

        if augment_mode == "mixup":
            x, ya, yb, lam = mixup_data(x, y, mixup_alpha)
            loss_fn = lambda logits: mixup_criterion(criterion, logits, ya, yb, lam)
        elif augment_mode == "cutmix":
            x, ya, yb, lam = cutmix_data(x, y, cutmix_alpha)
            loss_fn = lambda logits: mixup_criterion(criterion, logits, ya, yb, lam)
        else:
            loss_fn = lambda logits: criterion(logits, y)

        optimizer.zero_grad(set_to_none=True)
        # AMP (portable shim)
        with amp_autocast(enabled=use_amp):
            logits = model(x)
            loss = loss_fn(logits)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        if scheduler is not None:
            scheduler.step()

        bs = y.size(0)
        total += bs
        loss_sum += loss.item() * bs
    return loss_sum / total

#  main
def main():
    args = parse_args()

    # overrides from args
    overrides = {}
    if args.save_dir is not None: overrides["save_dir"] = args.save_dir
    if args.epochs is not None: overrides["epochs"] = args.epochs
    if args.batch_size is not None: overrides["batch_size"] = args.batch_size
    if args.lr is not None: overrides.setdefault("optimizer", {})["lr"] = args.lr
    if args.weight_decay is not None: overrides.setdefault("optimizer", {})["weight_decay"] = args.weight_decay
    if args.mixup_alpha is not None: overrides["mixup_alpha"] = args.mixup_alpha
    if args.cutmix_alpha is not None: overrides["cutmix_alpha"] = args.cutmix_alpha

    cfg = get_config(args.profile, overrides)
    set_seed(cfg["seed"])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ensure_dir(cfg["save_dir"])
    ckpt_dir = os.path.join(cfg["save_dir"], "ckpts")
    ensure_dir(ckpt_dir)
    log_f, writer = csv_logger(os.path.join(cfg["save_dir"], "metrics.csv"))

    mean, std = cfg["data"]["mean"], cfg["data"]["std"]
    train_tfms, test_tfms = build_transforms(mean, std)

    # CIFAR-10 datasets
    train_set = torchvision.datasets.CIFAR10(root=cfg["data"]["root"], train=True,  download=True, transform=train_tfms)
    test_set  = torchvision.datasets.CIFAR10(root=cfg["data"]["root"], train=False, download=True, transform=test_tfms)

    train_loader = DataLoader(train_set, batch_size=cfg["batch_size"], shuffle=True,
                              num_workers=cfg["num_workers"], pin_memory=True, drop_last=True)
    test_loader  = DataLoader(test_set,  batch_size=cfg["batch_size"], shuffle=False,
                              num_workers=cfg["num_workers"], pin_memory=True)

    # model / opt / loss / AMP / scheduler
    model = build_model(cfg["model"]["name"], num_classes=10).to(device)
    optimizer = build_optimizer(model, cfg)
    criterion = nn.CrossEntropyLoss(label_smoothing=cfg.get("label_smoothing", 0.0))
    scaler = make_scaler(enabled=cfg["amp"])  # AMP shim

    steps_per_epoch = len(train_loader)
    scheduler = build_warmup_cosine(optimizer, steps_per_epoch, cfg["epochs"], cfg["scheduler"]["warmup_epochs"])

    augment_mode = cfg.get("augment_mode", "off")
    mixup_alpha  = cfg.get("mixup_alpha", 0.0)
    cutmix_alpha = cfg.get("cutmix_alpha", 0.0)

    print(f"[cfg] profile={args.profile} | augment_mode={augment_mode} | mixup_alpha={mixup_alpha} | cutmix_alpha={cutmix_alpha} | save_dir={cfg['save_dir']}")

    best_acc = 0.0
    try:
        for epoch in range(cfg["epochs"]):
            tr_loss = train_one_epoch(model, train_loader, optimizer, scaler, device, criterion, scheduler,
                                      augment_mode, mixup_alpha, cutmix_alpha, cfg["amp"])
            va_loss, va_acc = evaluate(model, test_loader, device)
            writer.writerow({"epoch": epoch, "train_loss": tr_loss, "val_loss": va_loss, "val_acc": va_acc})
            if va_acc > best_acc:
                best_acc = va_acc
                save_ckpt({"epoch": epoch, "model": model.state_dict(), "best_acc": best_acc},
                          os.path.join(ckpt_dir, "best.pth"))
            print(f"epoch {epoch+1}/{cfg['epochs']} | train {tr_loss:.4f} | val {va_loss:.4f} | acc {va_acc*100:.2f}%")
    finally:
        log_f.close()

# entry — run MixUp, then CutMix in sequence (notebook-safe)
if __name__ == "__main__":
    # 1) MixUp
    os.environ["PROFILE"] = "mixup"
    print("\n==================== MIXUP RUN ====================")
    main()

    # 2) CutMix
    os.environ["PROFILE"] = "cutmix"
    print("\n==================== CUTMIX RUN ====================")
    main()



==================== MIXUP RUN ====================


100%|██████████| 170M/170M [00:14<00:00, 11.7MB/s]
/tmp/ipython-input-1464226345.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  return torch.cuda.amp.GradScaler(enabled=enabled)


[cfg] profile=mixup | augment_mode=mixup | mixup_alpha=0.4 | cutmix_alpha=0.0 | save_dir=results/mixup


train:   0%|          | 0/390 [00:00<?, ?it/s]/tmp/ipython-input-1464226345.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast(enabled=enabled)


KeyboardInterrupt: 

In [ ]:
# ==========================================================
# Summarizing MixUp vs CutMix results + Softmax Visualization + CSV export
# ==========================================================
import os, glob, pandas as pd, torch, torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as T

# ---- assumes build_model(...) is already defined elsewhere in your notebook ----
# it should accept model_name="resnet18_cifar" and num_classes=10 (as in training)

def load_runs(root):
    runs = []
    for csv_path in glob.glob(os.path.join(root, "**", "metrics.csv"), recursive=True):
        try:
            df = pd.read_csv(csv_path)
        except Exception:
            continue
        if len(df) == 0:
            continue
        runs.append({
            "run": os.path.dirname(csv_path),
            "final_acc": float(df["val_acc"].iloc[-1])
        })
    return pd.DataFrame(runs)

def _load_model_for_eval(model_ckpt_path, device, model_name="resnet18_cifar", num_classes=10):
    # Build the same model used in training; fallback if needed
    try:
        model = build_model(model_name, num_classes=num_classes).to(device)
    except Exception:
        # last-resort fallback: try plain torchvision resnet18 head for CIFAR10
        from torchvision.models import resnet18
        import torch.nn as nn
        model = resnet18(weights=None, num_classes=num_classes)
        # CIFAR conv1 tweak if your train-time model used 3x3 kernel/stride1
        if getattr(model, "conv1", None) and model.conv1.kernel_size != (3,3):
            model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        model = model.to(device)

    state = torch.load(model_ckpt_path, map_location=device)
    model.load_state_dict(state["model"])
    model.eval()
    return model

def _collect_max_softmax_confidences(model, testloader, device):
    confidences = []
    with torch.no_grad():
        for x, _ in testloader:
            x = x.to(device)
            probs = F.softmax(model(x), dim=1)
            max_conf, _ = probs.max(dim=1)
            confidences.extend(max_conf.detach().cpu().numpy())
    return confidences

def summarize(roots, *, model_name="resnet18_cifar", out_dir="results"):
    # ---- 1) Aggregate runs into tables ----
    parts = []
    for r in roots:
        tag = os.path.basename(r.rstrip("/"))
        df = load_runs(r)
        if df.empty:
            print(f"no runs found under {r}")
            continue
        df["tag"] = tag
        parts.append(df)

    if not parts:
        print("no runs to summarize")
        return None

    out = pd.concat(parts, ignore_index=True)

    # Per-run table (for audit)
    per_run = out[["run", "final_acc", "tag"]].copy()

    # Summary table
    summary = out.groupby("tag")["final_acc"].agg(["mean", "std", "count"]).reset_index()

    # Display (std may be NaN if count=1; that is expected)
    display(summary.style.set_caption("MixUp vs CutMix Summary").format({"mean": "{:.4f}", "std": "{:.4f}"}))

    # ---- 2) Softmax Visualization (one best checkpoint per tag) ----
    print("\nGenerating Softmax confidence plots...")
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Use the same normalization you used in training; both of these are common for CIFAR-10.
    mean, std = (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
    test_tfms = T.Compose([T.ToTensor(), T.Normalize(mean, std)])
    testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=test_tfms)
    testloader = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False, num_workers=2)

    plt.figure(figsize=(8,5))
    for r in roots:
        tag = os.path.basename(r.rstrip("/"))
        ckpt = os.path.join(r, "ckpts", "best.pth")
        if not os.path.exists(ckpt):
            print(f"No checkpoint found for {tag}, skipping softmax plot.")
            continue

        model = _load_model_for_eval(ckpt, device, model_name=model_name, num_classes=10)
        confidences = _collect_max_softmax_confidences(model, testloader, device)
        plt.hist(confidences, bins=30, alpha=0.5, label=f"{tag}")

    plt.title("Softmax Confidence Distribution — MixUp vs CutMix")
    plt.xlabel("Predicted Probability (max softmax)")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    # ---- 3) CSV exports ----
    os.makedirs(out_dir, exist_ok=True)
    summary_csv = os.path.join(out_dir, "summary.csv")
    perrun_csv = os.path.join(out_dir, "final_accs.csv")
    summary.to_csv(summary_csv, index=False)
    per_run.to_csv(perrun_csv, index=False)
    print(f"\nSaved summary to: {summary_csv}")
    print(f"Saved per-run table to: {perrun_csv}")

    return per_run


In [ ]:
# Uses the same model name you trained: "resnet18_cifar"
summarize(["results/mixup", "results/cutmix"], model_name="resnet18_cifar", out_dir="results")


In [ ]:
#For statistical testing (see RMD)
import pandas as pd, os

mixup_df = pd.read_csv("results/mixup/metrics.csv")
cutmix_df = pd.read_csv("results/cutmix/metrics.csv")

mixup_df["tag"] = "mixup"
cutmix_df["tag"] = "cutmix"

combined = pd.concat([mixup_df, cutmix_df], ignore_index=True)
combined.to_csv("results/all_metrics.csv", index=False)
print("✅ Combined CSV saved to results/all_metrics.csv")



In [ ]:
from google.colab import files
files.download("results/all_metrics.csv")